<a href="https://colab.research.google.com/github/Poznyakova/-7/blob/main/%D0%94%D0%97_%E2%84%9612_%D0%9F%D0%BE%D0%B7%D0%BD%D1%8F%D0%BA%D0%BE%D0%B2%D0%B0_%D0%9E%D0%BB%D1%8C%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

1. Загружаем библиотеки

In [ ]:
import pandas as pd
import numpy as np

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession #to define a SparkSession

sc = SparkContext('local') #spark connection
spark = SparkSession(sc) # open spark_session

2. Загружаем данные при помощи spark.read.csv из приложенного файла.

In [ ]:
df = spark.read.options(sep=",", header=True, quote="").csv('iris.csv')

выводим первые 5 строк данных

In [ ]:
df.show(5)

Выводим колонки

In [ ]:
df.columns

Преобразуем названия колонок, создаем новый датафрейм

In [ ]:
newColumns = ["sepal_length","sepal_width","petal_length","petal_width","variety"]
df_new = df.toDF(*newColumns)
df_new.show(5)

In [ ]:
from pyspark.sql.functions import *
#from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, IntegerType, DateType

Удаляем двойные кавычки с первого до последнеего атрибута

In [ ]:
df_new = df_new.withColumn("sepal_length", regexp_replace(col("sepal_length"), '"', ''))\
    .withColumn("variety", regexp_replace(col("variety"), '"', ''))
df_new.show(5)

Определяем типы атрибутов

In [ ]:
df_new.printSchema()

Преобразуем в другой тип данных

In [ ]:
df_new = df_new \
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
    .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))

In [ ]:
df_new.printSchema()

Итоговый датафрейм

In [ ]:
df_new.show(5)

3. Воспользуйтесь командами groupBy, max, min и avg, чтобы вывести максимальное минимальное и среднее значение каждого из атрибутов
цветка ('sepal_length','sepal_width','petal_length','petal_width').

Статистические показатели датафрейма

In [ ]:
df_new.summary().show()

Группируем

In [ ]:
df_new.groupBy('variety').sum('sepal_length').show()

In [ ]:
df_new.columns[:-1]

Создаем функцию

In [ ]:
def agg_func(attr):
  df_new.groupBy("variety")  \
    .agg(  \
      sum(attr),  \
      avg(attr),  \
      min(attr),  \
      max(attr))  \
    .show()

In [ ]:
for i in df_new.columns[:-1]:
  agg_func(i)

4. Постройте точечную диаграмму для petal_length и petal_width, раскрашенный по типам цветков.

Импортируем библиотеку seaborn

In [ ]:
import seaborn as sns

Преобразуем в тип датафрейма Pandas

In [ ]:
df_new_sample = df_new.toPandas()


Строим график

In [ ]:
sns.scatterplot(data = df_new_sample, x = "petal_length", y = "petal_width", hue = "variety")
plt.show()

5. Посмотрите внимательно на получившийся график и сформируйте правило вида ЕСЛИ ЗНАЧЕНИЕ ПАРАМЕТРА A < ЗНАЧЕНИЕ, ТО IRIS ОТНОСИТСЯ К КЛАССУ SETOSA - ИНАЧЕ К КАКОМУ-ТО ДРУГОМУ (НАЗОВЕМ, ЕГО НАПРИМЕР UNKNOWN).



Если значение petal_length < 2.5 и petal_width < 1, то это класс Setosa, иначе - Unknown

6. При помощи команд withColumn, when, otherwise создайте в датафрейме новую колонку с таким предсказанием.

In [ ]:
df_new_ex = df_new.withColumn("predict", \
                           when((df_new.petal_length < 2.5) & (df_new.petal_width < 1), lit("Setosa")) \
                           .otherwise(lit("Unknown"))
                           )

In [ ]:
df_new_ex.show(5)

7. Проверьте качество своего предсказания.

Для этого:

7.1. Подсчитайте количество строк, где Вы предсказали класс Setosa и
правильный класс был так же Setosa.

7.2. Подсчитайте количество строк, где Вы предсказали класс Setosa, а
правильный класс был какой-то другой (воспользуйтесь оператором "не
равно" - !=)

In [ ]:
df_new_ex.agg(
    count(when(col("variety") == "Setosa", True)), #сколько всего вариантов Setosa
    count(when((col("variety") == "Setosa") & (col("predict") == "Setosa"), True)), #сколько совпадений
    count(when((col("variety") != "Setosa") & (col("predict") == "Setosa"), True)) #сколько несовпадений
    ).show()